In [108]:
import numpy as np
import pickle
from nonparametric_linear_regression import construct_cluster_list

In [109]:
with open("US_covid_ar_3y/long_mem_gamma_clusters.pkl", "rb") as f:
    clusters_load = pickle.load(f)

In [110]:
with open("US_covid_ar_3y/long_mem_gamma_cluster_values.pkl", "rb") as f:
    cluster_values_load = pickle.load(f)

In [111]:
params = np.load("US_covid_ar_3y/long_mem_gamma_single_params.npz")

In [112]:
X = params["X"]
y = params["y"]
n, p = X.shape

In [113]:
cluster_values = cluster_values_load[2000:]

In [114]:
clusters = [construct_cluster_list(cluster_graph, n) for cluster_graph in clusters_load[2000:]]

In [124]:
def calc_acc(clusters, n):
    num = [len(x) for x in clusters]
    index = np.argpartition(num, len(num)-2)[-2:]
    first = min(index)
    second = max(index)

    count = 0

    for i in clusters[first]:
        if i >= 50:
            count +=1

    if count >25:
        temp = first
        first = second
        second = temp

    count = 0
    for i in clusters[first]:
        if i < 50:
            count += 1

    for i in clusters[second]:
        if i >= 50:
            count += 1

    return count/n

In [125]:
calc_acc(clusters[0], n)

0.93

In [126]:
clusters[0]

[[0,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 [1, 2, 3, 4, 5, 22],
 [6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49]]

In [127]:
cluster_acc = [calc_acc(cluster, n) for cluster in clusters]

In [128]:
cluster_length = [len(cluster) for cluster in clusters]

In [130]:
def calc_cluster_value(clusters, cluster_values):
    num = [len(x) for x in clusters]
    index = np.argpartition(num, len(num)-2)[-2:]
    first = min(index)
    second = max(index)

    count = 0

    for i in clusters[first]:
        if i >= 50:
            count +=1

    if count >25:
        temp = first
        first = second
        second = temp

    return cluster_values[first], cluster_values[second]

In [131]:
beta1 = np.zeros((len(cluster_values), 2))
beta2 = np.zeros((len(cluster_values), 2))

In [132]:
for i in range(len(cluster_values)):
    beta1[i], beta2[i] = calc_cluster_value(clusters[i], cluster_values[i])

In [133]:
print(f"cluster acc: {np.mean(cluster_acc)} +- {np.std(cluster_acc)}")
print(f"cluster length: {np.mean(cluster_length)} +- {np.std(cluster_length)}")
ls = {"beta1": beta1, "beta2": beta2}
for name, vec in ls.items():
    center = (np.quantile(vec, 0.025, axis = 0) + np.quantile(vec, 0.975, axis = 0))*0.5
    dist = center - np.quantile(vec, 0.025)
    print(f"{name}: {center} +- {dist}")

cluster acc: 0.9465412500000001 +- 0.08807354624651773
cluster length: 2.4195 +- 0.6314821850218738
beta1: [1.00044667 0.96897845] +- [3.36916239 3.33769417]
beta2: [1.92822037 1.63953372] +- [3.00779248 2.71910584]
